# Things to do: 
- What is CDD and HDD and what does it do?
- Lag Variable function updates
- Plotting total demand by 30min and group by week day
- Outlier handling: How we want to approach outliers found (Trimming, Capping, Discretization) 
- Plot demand vs season

In [1]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
# pd.options.display.max_columns = 50
# pd.options.display.max_rows = 50
# pd.options.display.width = 120
# pd.options.display.float_format = '{:.2f}'.format

# Loading Data
Loading data from the processed and combined csv file into the dataframe to commence preprocessing and cleansing.

In [4]:
source_data = r'./../data/NSW/processed_data.csv'  
# todo: this comes from my earlier stuff, need to add this work to that. no point in having 2 files

In [5]:
source_df = pd.read_csv(source_data).set_index('Unnamed: 0')
source_df

Checking if there are any NA values that we need to take into considering or drop the columns completely since data has been smoothed.

In [6]:
source_df.isna().sum()

In [7]:
source_df.columns

In [8]:
source_df.dtypes

In [9]:
updated_df = source_df

In [10]:
updated_df.index

In [11]:
updated_df.index=pd.to_datetime(updated_df.index)

Dropped Forecast_Daily and TotalDemand_daily - contained 33 N/A's.

In [12]:
updated_df.drop(['FORECASTDEMAND_daily', 'TOTALDEMAND_daily'], axis=1, inplace=True)

In [13]:
updated_df.isna().sum()

In [14]:
updated_df

In [15]:
updated_df.head()

# Feature Engineering

## DateTime Features
The following section creates date time features.

In [16]:
demand = updated_df.copy()
demand = demand[['totaldemand']]
demand.loc[:, 'dow'] = demand.index.dayofweek
demand.loc[:, 'doy'] = demand.index.dayofyear
demand.loc[:, 'year'] = demand.index.year
demand.loc[:, 'month'] = demand.index.month
demand.loc[:, 'quarter'] = demand.index.quarter
demand.loc[:, 'hour'] = demand.index.hour


In [17]:
demand.head()

In [18]:
demand.iloc[45: 60]

Merge to the updated_df

In [19]:
demand.isna().sum()

In [20]:
demand.index

In [21]:
final_df = pd.merge(updated_df, demand, left_index=True, right_index=True)

In [22]:
final_df

In [23]:
# final_df.isna().sum()

In [24]:
# final_df.index

## Adding Season Data
Season data being added to the dataframe before the final_df is created and exported to csv.

In [25]:
final_df['season'] = final_df['month'].apply(lambda month: 1 if month in [12, 1, 2] else
                                  (2 if month in [3, 4, 5] else
                                  (3 if month in [6, 7, 8] else
                                  (4 if month in [9, 10, 11] else None))))

In [26]:
final_df.index

In [27]:
final_df.head()

# Exporting Dataframe to CSV
The final dataframe is being exported to csv so that it can be used for additional analysis and modelling.

In [28]:
# final_df.to_csv(os.path.join('./../data/NSW', 'final_df.csv'))
# todo: this is what i have been working on but i need to add the later work

## Degree Days

In [29]:
def Degree_Days2(df, HDD_ct=17, CDD_ct=19.5):
    Tbar = df.resample('24H', offset='21H').mean() # Mean from 9pm (day i-1) - 9pm (day i)
    DD = pd.DataFrame(index=df.index, columns=['HDD', 'CDD'])
    for i in range(0,Tbar.shape[0]):
        DD['HDD'].iloc[48*(i):48*(i+1)] = max(0, HDD_ct-Tbar.iloc[i])
        DD['CDD'].iloc[48*(i):48*(i+1)] = max(0, Tbar.iloc[i]-CDD_ct)
    return Tbar, DD

In [30]:
Degree_Days2(final_df['TEMPERATURE'])

In [31]:
Tbar, DD = Degree_Days2(final_df['TEMPERATURE'])
# todo: is Tbar a new variable?

In [32]:
Tbar.isna().sum()

In [33]:
DD.isna().sum()

In [34]:
DD['HDD']

In [35]:
plt.plot(DD.iloc[1: 10000])
plt.title("Degree Days")
plt.xlabel("Time")
plt.ylabel("Degrees °C")
plt.legend()
plt.show()
# todo: fix warnings
# todo: are these new vars that need to be added to final_df?

# Outliers
Outlier detection is a method used to find unusual or abnormal data points in a data set. 
Methods of treating outliers:
- Trimming: removing the data from the dataset
- Capping:For instance, if we decide on a specific value, any data point above or below that value is considered an outlier. 
- Discretization: create groups and categorise the outliers into specific group making them follow the same behavior as the other points in that group.

## Temperature
The following section looks at the outlier identification within the Temperature column of our dataframe.

In [36]:
temp_df = final_df[['TEMPERATURE', 'month', 'season']]

### ScatterPlot of Temperature

In [37]:
plt.figure(figsize=(20, 10))
sns.scatterplot(data=temp_df, x='Unnamed: 0', y='TEMPERATURE', hue='season')
plt.title('Temperature ScatterPlot')
plt.xlabel('Date')
plt.ylabel('Temperature')
plt.show()

### Boxplot of Temperature
The boxplot is a method which allows for the easy visualisation of outliers within a given dataset. It can be seen that there are a number of data points within temperature which are at the very extreme and could be considered as outliers.

In [38]:
plt.figure(figsize=(20, 10))
sns.boxplot(data=temp_df, x='season', y='TEMPERATURE', hue='season')
plt.title('Box Plot of Temperature against seasons')
plt.xlabel('SEASONS')
plt.ylabel('TEMPERATURE')
plt.show()
# todo: is season a new variable?

In [39]:
plt.figure(figsize=(20,10))
sns.boxplot(data=temp_df[temp_df['season']==4], x='month', y='TEMPERATURE')
plt.title('Winter Months vs Temperature')
plt.show()

In [40]:
plt.figure(figsize=(20,10))
sns.boxplot(data=temp_df[temp_df['season']==1], x='month', y='TEMPERATURE')
plt.title('Summer Months vs Temperature')
plt.show()

In [41]:
plt.figure(figsize=(20,10))
sns.boxplot(data=temp_df[temp_df['season']==2], x='month', y='TEMPERATURE')
plt.title('Autumn Months vs Temperature')
plt.show()

In [42]:
plt.figure(figsize=(20,10))
sns.boxplot(data=temp_df[temp_df['season']==3], x='month', y='TEMPERATURE')
plt.title('Spring Months vs Temperature')
plt.show()

In [43]:
plt.figure(figsize=(20, 10))
sns.scatterplot(data=final_df[final_df['season']==1], x='Unnamed: 0', y='TEMPERATURE', hue='season')
plt.title('Temperature Outliers ScatterPlot')
plt.xlabel('Date')
plt.ylabel('Temperature')
plt.show()

## Temperature Outliers
The following sections looks further into the outliers for temperature which were identified in the temperature column of our datasets.

In [44]:
temp_highest_allowed = round(temp_df['TEMPERATURE'].mean() + 3 * temp_df['TEMPERATURE'].std(), 2)
temp_lowest_allowed = round(temp_df['TEMPERATURE'].mean() - 3 * temp_df['TEMPERATURE'].std(), 2)
print('Highest Allowed:', temp_highest_allowed)
print('Lowest Allowed:', temp_lowest_allowed)

In [45]:
temp_outliers = temp_df[(temp_df['TEMPERATURE']>temp_highest_allowed) | (temp_df['TEMPERATURE'] < temp_lowest_allowed)]
print('Total Rows:', len(temp_outliers))

In [46]:
plt.figure(figsize=(20, 10))
sns.scatterplot(data=temp_outliers, x='Unnamed: 0', y='TEMPERATURE', hue='season')
plt.title('Temperature Outliers ScatterPlot')
plt.xlabel('Date')
plt.ylabel('Temperature')
plt.show()

In [47]:
temp_month_df = temp_df[[]]
# fix_me: what was the plan here? 

### High Temperature Values
Outliers were identified for temperature values and high temperature outliers are more frequent from Nov to March. Indicating that during this period predictions may be less accurate.

In [48]:
high_outliers = temp_outliers.loc[temp_outliers['TEMPERATURE'] > temp_highest_allowed]
t_high_df = high_outliers['month'].value_counts()
t_high_df = t_high_df.reset_index()
t_high_df.columns = ['Month','Count']
t_high_df_sorted = t_high_df.sort_values(by='Month', ascending=True)
# print(test_df_sorted)
# fix_me: problem here with undefined dfs

In [49]:
# histogram
plt.figure(figsize=(15, 10))
sns.barplot(data=t_high_df_sorted, x='Month', y='Count')
plt.title('Distribution of High Temperature Outliers')
plt.show()

### Low Temperature Values
Lower tempature variations are far less those outliers observed for higher temperature variations. Lower tempature values can be found in June and July (Winter periods).

In [50]:
low_outliers = temp_outliers.loc[temp_outliers['TEMPERATURE'] < temp_lowest_allowed]
t_low_df = low_outliers['month'].value_counts()
t_low_df = t_low_df.reset_index()
t_low_df.columns = ['Month','Count']
t_low_df_sorted = t_low_df.sort_values(by='Month', ascending=True)
print(t_low_df_sorted)

In [51]:
#Histogram
plt.figure(figsize=(15,10))
sns.barplot(data=t_low_df, x='Month', y='Count')
plt.title('Distribution of Low Temperature Outliers')
plt.show()

Now that we know what the upper and lower caps are, we can apply capping to the outliers. This effectively means that we replace any values which exceed the upper and lower limits are replaced with the upper and lower limit respectively.

In [52]:
# final_df['TEMPERATURE'] = np.where(final_df['TEMPERATURE']>temp_highest_allowed,temp_highest_allowed, np.where(final_df['TEMPERATURE']<temp_lowest_allowed,temp_lowest_allowed,final_df['TEMPERATURE']))

In [53]:
# final_df_temp_highest = round(final_df['TEMPERATURE'].mean() + 3*final_df['TEMPERATURE'].std(),2)
# final_df_temp_lowest = round(final_df['TEMPERATURE'].mean() - 3*final_df['TEMPERATURE'].std(),2)
# print(final_df_temp_highest)
# print(final_df_temp_lowest)

## Price
The following sections looks at the outliers which might exist in the price data points which we are going to be using for future modelling.

In [54]:
price_outlier_df = final_df[['rrp', 'month', 'season']]
# print(price_outlier_df)

### Scatterplot of Price

In [55]:
plt.figure(figsize=(20, 10))
sns.scatterplot(data=price_outlier_df, x='Unnamed: 0', y='rrp', hue='season')
plt.title('Price ScatterPlot')
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()

### Price Outlier Analysis
Copying the values which are identified as outliers to a dataframe to analyse a little further.

In [56]:
price_highest_allowed = round(price_outlier_df['rrp'].mean() + 3 * price_outlier_df['rrp'].std(), 2)
price_lowest_allowed = round(price_outlier_df['rrp'].mean() - 3 * price_outlier_df['rrp'].std(), 2)

In [57]:
price_outliers = price_outlier_df[(price_outlier_df['rrp']>price_highest_allowed) | (price_outlier_df['rrp']<price_lowest_allowed)]
print(len(price_outliers))

In [58]:
price_outliers

In [59]:
plt.figure(figsize=(20, 10))
sns.scatterplot(data=price_outliers,x='month', y='rrp', hue='season')
plt.title('Scatterplot of Price by Month')
plt.legend()
plt.show()

#### Price Outlier by Month

In [60]:
# low_outliers = temp_outliers.loc[temp_outliers['TEMPERATURE']<temp_lowest_allowed]
price_out_df = price_outliers['month'].value_counts()
price_out_df = price_out_df.reset_index()
price_out_df.columns = ['Month', 'Count']
price_out_df_sorted = price_out_df.sort_values(by='Month', ascending=True)
# print(price_out_df_sorted)

In [61]:
# Histogram
plt.figure(figsize=(15, 10))
sns.barplot(data=price_out_df_sorted, x='Month', y='Count')
plt.title('Distribution of Price Outliers')
plt.show()

## Total Demand

In [62]:
total_demand_highest_allowed = round(final_df['TOTALDEMAND'].mean() + 3 * final_df['TOTALDEMAND'].std(), 2)
total_demand_lowest_allowed = round(final_df['TOTALDEMAND'].mean() - 3 * final_df['TOTALDEMAND'].std(), 2)
print(total_demand_highest_allowed)
print(total_demand_lowest_allowed)

In [63]:
total_demand_outliers = final_df[(final_df['TOTALDEMAND'] > total_demand_highest_allowed) | (final_df['TOTALDEMAND'] < total_demand_lowest_allowed)]
print(len(total_demand_outliers))

In [64]:
plt.figure(figsize=(20, 10))
plt.scatter(
    total_demand_outliers.index, 
    total_demand_outliers['TOTALDEMAND'], 
    c=total_demand_outliers['month']
)
plt.legend()
plt.show()

In [65]:
print(total_demand_outliers['month'].value_counts())
#ToDO: Sort values by month

Total Demand outliers can be seen during the months from Nov to Feb. 

In [66]:
from sklearn.ensemble import IsolationForest

In [67]:
handle_outliers = final_df[['TEMPERATURE', 'rrp', 'TOTALDEMAND', 'month', 'season']]

In [68]:
random_state = np.random.RandomState(42)

model = IsolationForest(
    n_estimators=100, 
    max_samples='auto', 
    contamination=float(0.003)
)

model.fit(handle_outliers[['TEMPERATURE']])
print(model.get_params())
handle_outliers['Iso_forest_scores'] = model.decision_function(handle_outliers[['TEMPERATURE']])
handle_outliers['anomaly_score'] = model.predict(handle_outliers[['TEMPERATURE']])
handle_outliers[handle_outliers['anomaly_score'] == -1].head()
anomaly_df = handle_outliers[handle_outliers['anomaly_score'] == -1]
no_anomaly_df = handle_outliers[handle_outliers['anomaly_score'] == 1]

In [69]:
anomaly_df.columns

In [70]:
anomaly_df

In [71]:
print('Total Anomalies:', len(anomaly_df))
print('Total non-Anomaly:', len(no_anomaly_df))

## Temperature Data - Checking Observations per Day
Where the dates were grouped, the number of observations were counted and if the number of expected observations is less than 48 the rows related to this date are dropped.
- 5 minute intervals = (24 * 60)/5 = 288 observations (per day)
- 30 minute intervals = (24*60)/30 = 48 observations (per day)

30 days of data are missing from the observations.
30 days -> 30*48 = 1440

Final_df
- Has this been cleaned and outlier handling been done

Check temperature data for missing observations.


In [155]:
temp_data = r'./../data/NSW/temperature_nsw.csv' 
temp = pd.read_csv(temp_data)
print(temp.shape[0])
temp = temp[['DATETIME','TEMPERATURE']]
temp.index = pd.to_datetime(temp.index)
temp.index = temp.index.strftime('%Y-%m-%d')

temp.set_index('DATETIME',inplace=True)
temp.index = pd.to_datetime(temp.index)

In [124]:
temp['date'] = temp.index.date
temp['time'] = temp.index.time

#Sorting index to get days of months together - previously sorted by month
temp.sort_index(ascending=True,inplace=True)

temp_resampled = temp.resample('30min').mean()
print(temp_resampled.shape[0])
print(temp_resampled.tail())



In [169]:
temp_obs = temp_resampled.groupby(temp_resampled.index.date).size().reset_index(name='obs_count')
temp_obs.set_index('index',inplace=True)
print(temp_obs)

In [170]:
temp_obs.index = pd.to_datetime(temp_obs.index)
temp_obs = temp_obs[temp_obs.index == '2013-02-02']
print(temp_obs)

In [171]:
print(temp_obs[temp_obs['obs_count']>48])

We are only missing days of data where complete observations are missing - there are consistently 48 readings per day.

In [172]:
temp_df = final_df[['TEMPERATURE', 'month', 'season']]
print(temp_df.head())

In [173]:
test_obs = temp_df.groupby(temp_df.index.date).size().reset_index(name='obs_count')
print(test_obs)

In [174]:
test_obs = test_obs[test_obs['obs_count']<48]
print(test_obs.head(161))
print(test_obs.shape[0])

There are some differences for dates between the raw temp data and final_df. This could be attributed to data loss during condolidation of dataframes, there are 161 impacted dates.